In [1]:
from active_critic.utils.gym_utils import *
from gym.spaces.box import Box

/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [2]:
class MultiEnvWrapper(gym.Wrapper):
    def __init__(self, list_envs) -> None:
        super().__init__(list_envs[0])
        self.list_envs = list_envs
        self.current_env = 0
        #self._make_observation_space()


    def reset(self):
        self.current_env = (self.current_env + 1) % len(self.list_envs)
        #self._make_observation_space()
        obs =  super().reset()
        #obs = np.concatenate((obs[:-4], np.array([self.current_env]), obs[-4:]))
        return obs

    def step(self, action):

        obs, rew, done, info = super().step(action)
        #obs = np.append(obs, self.current_env)
        #obs = np.concatenate((obs[:-4], np.array([self.current_env]), obs[-4:]))

        return obs, rew, done, info

    def _make_observation_space(self):
        low = self.list_envs[self.current_env].observation_space.low
        high = self.list_envs[self.current_env].observation_space.high
        dtype = self.list_envs[self.current_env].observation_space.dtype
        new_low = np.concatenate((low[:-4], np.array([0]), low[-4:]))
        new_high = np.concatenate((high[:-4], np.array([len(self.list_envs)]), high[-4:]))
        new_obs_space = Box(new_low, new_high, dtype=dtype)
        self.observation_space = new_obs_space
        self.action_space = self.list_envs[self.current_env].action_space

def make_env_list(env_ids):
    policy_dict = make_policy_dict()
    list_envs = []
    list_experts = []
    for i in range(len(env_ids)):
        exp, env_id = policy_dict[env_ids[i]]
        env = ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE[env_id]()
        env._freeze_rand_vec = False
        list_envs.append(env)
        list_experts.append(exp)
    return list_envs, list_experts
    
class MultiImitationLearningWrapper:
    def __init__(self, policies, env: GymEnv):
        self.observation_space = env.observation_space
        self.action_space = env.action_space
        self.policies = policies

    def predict(self, obsv, deterministic=None):
        actions = []
        for obs in obsv:
            current_env = int(obs[-4])
            actions.append(self.policies[current_env].get_action(obs))
        return actions

In [3]:
def make_multi_vec_env(env_ids, num_cpu, seq_len, sparse):

    def make_env(env_ids, rank, seed=0):
        def _init():
            max_episode_steps = seq_len
            list_envs, list_experts = make_env_list(env_ids)
            #multi_env = MultiEnvWrapper(list_envs=list_envs)
            multi_env = list_envs[0]
            timelimit = TimeLimit(env=multi_env, max_episode_steps=max_episode_steps)
            strict_time = StrictSeqLenWrapper(timelimit, seq_len=seq_len + 1, sparse=sparse)
            riw = RolloutInfoWrapper(strict_time)
            return riw
        return _init
        
    env = SubprocVecEnv([make_env(env_ids, i) for i in range(num_cpu)])

    list_envs, list_experts = make_env_list(env_ids)
    vec_expert = MultiImitationLearningWrapper(
        policies=list_experts, env=env)
    return env, vec_expert

In [4]:
vec_env, exp = make_dummy_vec_env(name='reach', seq_len=100, sparse=False)

/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [ ]:
obs = vec_env.reset()
done = False
while not done:
    actions = exp.predict(obs)
    ons, rew, dones, info = vec_env.step(actions)
    done = dones[0]
    print(rew)
#print(rew)

In [ ]:
actions, observations, rewards, _, expected_rewards, _ = sample_new_episode(
            policy=exp,
            env=vec_env,
            dense=True,
            extractor=DummyExtractor(),
            device='cpu',
            episodes=2,
            seq_len=100)

In [5]:
transitions = sample_expert_transitions(
            exp.predict, vec_env, 2)

Sampling transitions. 2
obsv: [ 0.00615235  0.6001898   0.19430117  1.         -0.04175089  0.65508085
  0.02        0.          0.          0.          1.          0.
  0.          0.          0.          0.          0.          0.
  0.00615235  0.6001898   0.19430117  1.         -0.01229408  0.64236444
  0.02        0.          0.          0.          1.          0.
  0.          0.          0.          0.          0.          0.
  0.08323025  0.8700931   0.10802165]
obsv: [ 6.1291843e-03  6.0036105e-01  1.9390731e-01  1.0000000e+00
 -4.1753512e-02  6.5508449e-01  1.9923743e-02 -9.8167395e-05
 -7.0607253e-05 -3.2230965e-09  1.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  6.1523519e-03  6.0018981e-01
  1.9430117e-01  1.0000000e+00 -4.1750893e-02  6.5508085e-01
  2.0000000e-02  0.0000000e+00  0.0000000e+00  0.0000000e+00
  1.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  

/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/metaworld/policies/policy.py:41: UserWarning: Constant(s) may be too high. Environments clip response to [-1, 1]
  warnings.warn('Constant(s) may be too high. Environments clip response to [-1, 1]')


In [ ]:
transitions.infos

In [ ]:
rewards

In [6]:
multi_vec, multi_exp = make_multi_vec_env(env_ids=['reach'], num_cpu=2, seq_len=100, sparse=False)

/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [8]:
obs = multi_vec.reset()
done = False
while not done:
    actions = multi_exp.predict(obs)
    ons, rew, dones, info = multi_vec.step(actions)
    done = dones[0]
    print(rew)
#print(rew)

[1.43555554 1.65361219]
[1.45320918 1.67477844]
[1.48959177 1.72178767]
[1.54542994 1.79912461]
[1.62066652 1.91010824]
[1.71515645 2.05777089]
[1.82892493 2.24527215]
[1.96196945 2.45393711]
[2.10844304 2.6814907 ]
[2.26553859 2.94051209]
[2.43571702 3.24033243]
[2.62280787 3.58744017]
[2.83055113 3.98586208]
[3.06204648 4.43689028]
[3.31951048 4.9382033 ]
[3.6041528  5.48278397]
[3.9161286 6.0581384]
[4.25454309 6.64633364]
[4.61748092 7.22526689]
[5.00203743 7.77126506]
[5.40434215 8.26260548]
[5.81958323 8.68304277]
[6.24205805 9.02423762]
[6.66528569 9.28768211]
[7.08221418 9.49726323]
[7.48553984 9.67396696]
[7.86812858 9.81431388]
[8.2234974  9.91260354]
[8.54628932 9.97028851]
[8.83266484 9.995626  ]
[ 9.08054422 10.        ]
[ 9.28966306 10.        ]
[ 9.46144071 10.        ]
[ 9.59869437 10.        ]
[ 9.70525296 10.        ]
[ 9.78553059 10.        ]
[ 9.84411238 10.        ]
[ 9.88539176 10.        ]
[9.91328351 9.99994405]
[9.9310243  9.99142392]
[9.93638321 9.96317189]
[9

In [ ]:
obs = multi_vec.reset()

In [ ]:
obs

In [ ]:
multi_vec.reset()

In [ ]:
b = np.concatenate((a, np.array([100]), a[10:]))

In [ ]:
b